# உரை வகைப்படுத்தல் பணிகள்

இந்த தொகுதியில், **[AG_NEWS](http://www.di.unipi.it/~gulli/AG_corpus_of_news_articles.html)** தரவுத்தொகுப்பை அடிப்படையாகக் கொண்ட ஒரு எளிய உரை வகைப்படுத்தல் பணியைத் தொடங்குவோம்: செய்தி தலைப்புகளை உலகம், விளையாட்டு, வணிகம் மற்றும் அறிவியல்/தொழில்நுட்பம் ஆகிய 4 வகைகளில் ஒன்றாக வகைப்படுத்துவோம்.

## தரவுத்தொகுப்பு

தரவுத்தொகுப்பை ஏற்ற, **[TensorFlow Datasets](https://www.tensorflow.org/datasets)** APIயைப் பயன்படுத்துவோம்.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

# In this tutorial, we will be training a lot of models. In order to use GPU memory cautiously,
# we will set tensorflow option to grow GPU memory allocation when required.
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

dataset = tfds.load('ag_news_subset')

நாம் இப்போது `dataset['train']` மற்றும் `dataset['test']` ஆகியவற்றைப் பயன்படுத்தி தரவுத்தொகுப்பின் பயிற்சி மற்றும் சோதனை பகுதிகளை அணுகலாம்:


In [3]:
ds_train = dataset['train']
ds_test = dataset['test']

print(f"Length of train dataset = {len(ds_train)}")
print(f"Length of test dataset = {len(ds_test)}")

Length of train dataset = 120000
Length of test dataset = 7600


நாம் எங்கள் தரவுத்தொகுப்பில் இருந்து முதல் 10 புதிய தலைப்புகளை அச்சிடலாம்:


In [4]:
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

for i,x in zip(range(5),ds_train):
    print(f"{x['label']} ({classes[x['label']]}) -> {x['title']} {x['description']}")

3 (Sci/Tech) -> b'AMD Debuts Dual-Core Opteron Processor' b'AMD #39;s new dual-core Opteron chip is designed mainly for corporate computing applications, including databases, Web services, and financial transactions.'
1 (Sports) -> b"Wood's Suspension Upheld (Reuters)" b'Reuters - Major League Baseball\\Monday announced a decision on the appeal filed by Chicago Cubs\\pitcher Kerry Wood regarding a suspension stemming from an\\incident earlier this season.'
2 (Business) -> b'Bush reform may have blue states seeing red' b'President Bush #39;s  quot;revenue-neutral quot; tax reform needs losers to balance its winners, and people claiming the federal deduction for state and local taxes may be in administration planners #39; sights, news reports say.'
3 (Sci/Tech) -> b"'Halt science decline in schools'" b'Britain will run out of leading scientists unless science education is improved, says Professor Colin Pillinger.'
1 (Sports) -> b'Gerrard leaves practice' b'London, England (Sports Network

## உரை வெக்டரமயமாக்கல்

இப்போது உரையை **எண்களாக** மாற்ற வேண்டும், அவை டென்சர்களாக பிரதிநிதித்துவப்படுத்தப்பட வேண்டும். நாம் சொற்-நிலை பிரதிநிதித்துவத்தை விரும்பினால், இரண்டு விஷயங்களை செய்ய வேண்டும்:

* உரையை **டோக்கன்களாக** பிரிக்க **டோக்கனைசரை** பயன்படுத்த வேண்டும்.
* அந்த டோக்கன்களின் **சொற்களஞ்சியத்தை** உருவாக்க வேண்டும்.

### சொற்களஞ்சிய அளவை வரையறுத்தல்

AG News தரவுத்தொகுப்பின் உதாரணத்தில், சொற்களஞ்சியத்தின் அளவு மிகவும் பெரியதாக உள்ளது, 100,000 க்கும் மேற்பட்ட சொற்கள். பொதுவாக, உரையில் அரிதாகவே காணப்படும் சொற்கள் தேவையில்லை &mdash; சில வாக்கியங்களில் மட்டுமே அவை இருக்கும், மேலும் மாடல் அவற்றிலிருந்து கற்றுக்கொள்ளாது. எனவே, வெக்டரைசர் கட்டமைப்பாளருக்கு ஒரு வாதத்தை வழங்குவதன் மூலம் சொற்களஞ்சியத்தின் அளவை குறைக்க நியாயமாகும்:

இந்த இரண்டு படிகளையும் **TextVectorization** அடுக்கு மூலம் கையாளலாம். முதலில் வெக்டரைசர் பொருளை உருவாக்கி, பின்னர் `adapt` முறைமையை அழைத்து அனைத்து உரையையும் கடந்து சொற்களஞ்சியத்தை உருவாக்கலாம்:


In [5]:
vocab_size = 50000
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size)
vectorizer.adapt(ds_train.take(500).map(lambda x: x['title']+' '+x['description']))

> **குறிப்பு** நாம் முழு தரவுத்தொகுப்பின் ஒரு பகுதியை மட்டுமே சொற்களஞ்சியத்தை உருவாக்க பயன்படுத்துகிறோம். இதைச் செய்வதன் மூலம் செயல்பாட்டு நேரத்தை வேகமாக்கி, உங்களை காத்திருக்காமல் செய்கிறோம். ஆனால், முழு தரவுத்தொகுப்பிலிருந்து சில சொற்கள் சொற்களஞ்சியத்தில் சேர்க்கப்படாமல் போகும் அபாயத்தை நாம் ஏற்கிறோம், மேலும் அவை பயிற்சியின் போது புறக்கணிக்கப்படும். எனவே, `adapt` செயல்பாட்டின் போது முழு சொற்களஞ்சிய அளவையும் பயன்படுத்தி, முழு தரவுத்தொகுப்பைச் சுழற்றுவது இறுதி துல்லியத்தை அதிகரிக்கலாம், ஆனால் அது மிகுந்த அளவில் இல்லை.

இப்போது நாங்கள் உண்மையான சொற்களஞ்சியத்தை அணுக முடியும்:


In [6]:
vocab = vectorizer.get_vocabulary()
vocab_size = len(vocab)
print(vocab[:10])
print(f"Length of vocabulary: {vocab_size}")

['', '[UNK]', 'the', 'to', 'a', 'in', 'of', 'and', 'on', 'for']
Length of vocabulary: 5335


வெக்டரைசரைப் பயன்படுத்தி, எந்த உரையையும் எளிதாக எண்களின் தொகுப்பாக குறியாக்கலாம்:


In [7]:
vectorizer('I love to play with my words')

<tf.Tensor: shape=(7,), dtype=int64, numpy=array([ 112, 3695,    3,  304,   11, 1041,    1], dtype=int64)>

## Bag-of-words உரை பிரதிநிதித்துவம்

சொற்கள் அர்த்தத்தை பிரதிநிதித்துவப்படுத்துவதால், சில நேரங்களில் ஒரு உரையின் அர்த்தத்தை அதன் சொற்களை மட்டும் பார்த்து, அவை வாக்கியத்தில் எந்த வரிசையில் உள்ளன என்பதை பொருட்படுத்தாமல் கண்டறிய முடியும். உதாரணமாக, செய்திகளை வகைப்படுத்தும்போது, *வானிலை* மற்றும் *பனி* போன்ற சொற்கள் *வானிலை முன்னறிவிப்பு* என்பதை சுட்டிக்காட்டும், அதே நேரத்தில் *பங்கு* மற்றும் *டாலர்* போன்ற சொற்கள் *நிதி செய்திகள்* என்பதற்காக கணக்கில் எடுத்துக்கொள்ளப்படும்.

**Bag-of-words** (BoW) வெக்டர் பிரதிநிதித்துவம் பாரம்பரிய வெக்டர் பிரதிநிதித்துவங்களில் மிகவும் எளிதாக புரிந்துகொள்ளக்கூடியது. ஒவ்வொரு சொல்லும் ஒரு வெக்டர் குறியீட்டுடன் இணைக்கப்பட்டிருக்கும், மேலும் ஒரு வெக்டர் கூறு ஒரு குறிப்பிட்ட ஆவணத்தில் ஒவ்வொரு சொல்லின் நிகழ்வுகளின் எண்ணிக்கையை கொண்டிருக்கும்.

![Bag-of-words வெக்டர் பிரதிநிதித்துவம் நினைவகத்தில் எப்படி பிரதிநிதித்துவப்படுத்தப்படுகிறது என்பதை காட்டும் படம்.](../../../../../translated_images/ta/bag-of-words-example.606fc1738f1d7ba9.webp) 

> **Note**: BoW-ஐ உரையில் உள்ள தனிப்பட்ட சொற்களுக்கான அனைத்து ஒரே-ஹாட்-கோடிடப்பட்ட வெக்டர்களின் கூட்டமாகவும் நீங்கள் சிந்திக்கலாம்.

கீழே Scikit Learn பைதான் நூலகத்தைப் பயன்படுத்தி Bag-of-words பிரதிநிதித்துவத்தை உருவாக்குவதற்கான ஒரு உதாரணம்:


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
sc_vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
sc_vectorizer.fit_transform(corpus)
sc_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

மேலே நாம் வரையறுத்த Keras வெக்டரைசரை பயன்படுத்தலாம், ஒவ்வொரு வார்த்தை எண்ணையும் ஒரு ஒன்-ஹாட் என்கோடிங்காக மாற்றி, அந்த வெக்டர்களை எல்லாம் சேர்க்கலாம்:


In [9]:
def to_bow(text):
    return tf.reduce_sum(tf.one_hot(vectorizer(text),vocab_size),axis=0)

to_bow('My dog likes hot dogs on a hot day.').numpy()

array([0., 5., 0., ..., 0., 0., 0.], dtype=float32)

> **குறிப்பு**: முந்தைய உதாரணத்திலிருந்து முடிவு மாறுபடுவது உங்களை ஆச்சரியப்படுத்தலாம். காரணம், Keras உதாரணத்தில் வெக்டரின் நீளம் முழு AG News தரவுத்தொகுப்பிலிருந்து உருவாக்கப்பட்ட சொற்கள்தொகையின் அளவுக்கு இணையாக இருக்கும், ஆனால் Scikit Learn உதாரணத்தில் நாம் மாதிரித் உரை அடிப்படையில் சொற்கள்தொகையை உடனடியாக உருவாக்கினோம்.


## BoW வகைப்பாட்டாளரை பயிற்சி செய்வது

இப்போது நமது உரையின் Bag-of-Words (BoW) பிரதிநிதித்துவத்தை உருவாக்குவது எப்படி என்பதை கற்றுக்கொண்டுள்ளோம், அதை பயன்படுத்தும் வகைப்பாட்டாளரை பயிற்சி செய்யலாம். முதலில், நமது தரவுத்தொகுப்பை Bag-of-Words பிரதிநிதித்துவமாக மாற்ற வேண்டும். இதை கீழே உள்ளபடி `map` செயல்பாட்டைப் பயன்படுத்தி செய்யலாம்:


In [11]:
batch_size = 128

ds_train_bow = ds_train.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)
ds_test_bow = ds_test.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)

இப்போது, ஒரு எளிய வகைப்பாட்டாளர் நரம்பியல் வலைப்பின்னலை வரையறுப்போம், இது ஒரு நேரியல் அடுக்கு கொண்டுள்ளது. உள்ளீட்டு அளவு `vocab_size`, மற்றும் வெளியீட்டு அளவு வகைகளின் எண்ணிக்கையை (4) குறிக்கிறது. நாம் ஒரு வகைப்பாட்டு பணியை தீர்க்கிறோம் என்பதால், இறுதி செயல்பாட்டு செயல்பாடு **softmax** ஆகும்:


In [12]:
model = keras.models.Sequential([
    keras.layers.Dense(4,activation='softmax',input_shape=(vocab_size,))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train_bow,validation_data=ds_test_bow)

938/938 [==============================] - 66s 70ms/step - loss: 0.6144 - acc: 0.8427 - val_loss: 0.4416 - val_acc: 0.8697


நாம் 4 வகுப்புகள் கொண்டுள்ளதால், 80% க்கும் மேற்பட்ட துல்லியத்துடன் ஒரு நல்ல முடிவாகும்.

## ஒரு நெட்வொர்க்காக வகுப்பாளரை பயிற்சி செய்வது

Vectorizer கூட Keras layer ஆக இருப்பதால், அதை உள்ளடக்கிய ஒரு நெட்வொர்க்கை வரையறுத்து, முழுமையாக பயிற்சி செய்யலாம். இவ்வாறு, `map` பயன்படுத்தி தரவுத்தொகுப்பை vectorize செய்ய தேவையில்லை, நெட்வொர்க்கின் உள்ளீட்டிற்கு அசல் தரவுத்தொகுப்பை நேரடியாக வழங்கலாம்.

> **குறிப்பு**: எங்கள் தரவுத்தொகுப்பில் உள்ள `title`, `description` மற்றும் `label` போன்ற புலங்களை dictionary களிலிருந்து tuples களாக மாற்ற maps ஐ இன்னும் பயன்படுத்த வேண்டும். ஆனால், டிஸ்கில் இருந்து தரவுகளை ஏற்றும்போது, தேவையான அமைப்புடன் ஒரு தரவுத்தொகுப்பை முதலில் உருவாக்கலாம்.


In [13]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

inp = keras.Input(shape=(1,),dtype=tf.string)
x = vectorizer(inp)
x = tf.reduce_sum(tf.one_hot(x,vocab_size),axis=1)
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
model.summary()

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 5335)        0         
                                                                 
 tf.math.reduce_sum (TFOpLam  (None, 5335)             0         
 bda)                                                            
                                                                 
 dense_2 (Dense)             (None, 4)                 21344     
                                                                 
Total params: 21,344
Trainable params: 21,344
Non-trainable p

## பைகிராம்கள், டிரைகிராம்கள் மற்றும் என்-கிராம்கள்

பேக்-ஆஃப்-வேர்ட்ஸ் அணுகுமுறையின் ஒரு வரம்பு என்னவென்றால், சில சொற்கள் பல சொற்களால் ஆன வெளிப்பாடுகளின் ஒரு பகுதியாக இருக்கின்றன. உதாரணமாக, 'ஹாட் டாக்' என்ற சொல், 'ஹாட்' மற்றும் 'டாக்' என்ற சொற்களுடன் வேறுபட்ட அர்த்தம் கொண்டது. 'ஹாட்' மற்றும் 'டாக்' என்ற சொற்களை எப்போதும் ஒரே வெக்டர்களைப் பயன்படுத்தி பிரதிநிதித்துவப்படுத்தினால், அது நமது மாதிரியை குழப்பமாக்கும்.

இதற்கு தீர்வாக, **என்-கிராம் பிரதிநிதித்துவங்கள்** ஆவண வகைப்பாட்டின் முறைகளில் பெரும்பாலும் பயன்படுத்தப்படுகின்றன. இதில் ஒவ்வொரு சொல், இரு-சொல் அல்லது மூன்று-சொல் அடர்த்தியும் வகைப்பாட்டாளர்களை பயிற்சி செய்ய பயனுள்ள அம்சமாக இருக்கும். உதாரணமாக, பைகிராம் பிரதிநிதித்துவங்களில், அசல் சொற்களுடன் சேர்த்து அனைத்து சொல் ஜோடிகளையும் அகராதியில் சேர்ப்போம்.

கீழே, Scikit Learn பயன்படுத்தி பைகிராம் பேக்-ஆஃப்-வேர்ட்ஸ் பிரதிநிதித்துவத்தை உருவாக்குவதற்கான ஒரு உதாரணம் கொடுக்கப்பட்டுள்ளது:


In [14]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

n-gram முறையின் முக்கிய குறைபாடு என்னவென்றால், சொற்களஞ்சியத்தின் அளவு மிகவும் வேகமாக அதிகரிக்கத் தொடங்குகிறது. நடைமுறையில், n-gram பிரதிநிதித்துவத்தை *embedding* போன்ற பரிமாணத்தை குறைக்கும் தொழில்நுட்பத்துடன் இணைக்க வேண்டும், இதைப் பற்றி நாம் அடுத்த அலகில் விவாதிக்கப் போகிறோம்.

**AG News** தரவுத்தொகுப்பில் n-gram பிரதிநிதித்துவத்தை பயன்படுத்த, `TextVectorization` கட்டமைப்பிற்கு `ngrams` அளவுருவை அனுப்ப வேண்டும். ஒரு bigram சொற்களஞ்சியத்தின் நீளம் **மிக அதிகமாக** இருக்கும், எங்கள் நிலைமையில் இது 1.3 மில்லியனுக்கும் மேற்பட்ட டோக்கன்களாகும்! எனவே, ஒரு நியாயமான எண்ணிக்கையால் bigram டோக்கன்களையும் வரையறுப்பது பொருத்தமாக இருக்கும்.

மேலே உள்ள அதே குறியீட்டை பயன்படுத்தி வகைப்பாட்டாளரை பயிற்றுவிக்கலாம், ஆனால் இது நினைவகத்தை மிகவும் செயல்திறனற்றதாக மாற்றும். அடுத்த அலகில், நாம் embedding-களைப் பயன்படுத்தி bigram வகைப்பாட்டாளரை பயிற்றுவிக்கப் போகிறோம். இதற்கிடையில், இந்த நோட்புக்கில் bigram வகைப்பாட்டாளர் பயிற்சியை நீங்கள் சோதித்து, அதிக துல்லியத்தைப் பெற முடியுமா என்று பார்க்கலாம்.


## BoW வெக்டர்களை தானாக கணக்கிடுதல்

மேலே உள்ள உதாரணத்தில், ஒவ்வொரு சொற்களின் ஒற்றை-ஹாட் குறியீடுகளை கூட்டுவதன் மூலம் BoW வெக்டர்களை கையால் கணக்கிட்டோம். ஆனால், TensorFlow இன் சமீபத்திய பதிப்பு BoW வெக்டர்களை தானாக கணக்கிட `output_mode='count` என்ற அளவுருவை vectorizer constructor-க்கு அனுப்புவதன் மூலம் எளிதாக்குகிறது. இது எங்கள் மாதிரியை வரையறுத்தல் மற்றும் பயிற்சி செய்வதை குறிப்பிடத்தக்க அளவில் எளிதாக்குகிறது:


In [15]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='count'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 7s 7ms/step - loss: 0.5929 - acc: 0.8486 - val_loss: 0.4168 - val_acc: 0.8772


## வார்த்தை அதிர்வெண் - ஆவண எதிர்மறை அதிர்வெண் (TF-IDF)

BoW பிரதிநிதித்துவத்தில், வார்த்தை தோற்றங்கள் எந்த வார்த்தை என்றாலும் ஒரே முறையில் எடைபோடப்படுகின்றன. ஆனால், *a* மற்றும் *in* போன்ற அடிக்கடி வரும் வார்த்தைகள் வகைப்படுத்தலுக்குப் பெரிதும் முக்கியமல்ல என்பது தெளிவாக உள்ளது. பெரும்பாலான இயற்கை மொழி செயலாக்க பணிகளில் சில வார்த்தைகள் மற்றவற்றைவிட அதிகம் தொடர்புடையவை.

**TF-IDF** என்பது **term frequency - inverse document frequency** என்பதற்கான சுருக்கமாகும். இது bag-of-words முறைமையின் ஒரு மாறுபாடு, இதில் ஒரு ஆவணத்தில் ஒரு வார்த்தையின் தோற்றத்தை குறிக்கும் பைனரி 0/1 மதிப்பின் பதிலாக, ஒரு மிதவை புள்ளி மதிப்பு பயன்படுத்தப்படுகிறது, இது அந்த வார்த்தையின் தொகுப்பில் தோன்றும் அதிர்வெண்களுடன் தொடர்புடையது.

முறைமையாக, ஒரு ஆவணத்தில் $i$ என்ற வார்த்தையின் எடை $w_{ij}$ இவ்வாறு வரையறுக்கப்படுகிறது:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
இதில்
* $tf_{ij}$ என்பது $j$ ஆவணத்தில் $i$ தோன்றும் எண்ணிக்கை, அதாவது நாம் முன்பு பார்த்த BoW மதிப்பு
* $N$ என்பது தொகுப்பில் உள்ள ஆவணங்களின் எண்ணிக்கை
* $df_i$ என்பது முழு தொகுப்பில் $i$ என்ற வார்த்தையை உள்ளடக்கிய ஆவணங்களின் எண்ணிக்கை

TF-IDF மதிப்பு $w_{ij}$ ஒரு ஆவணத்தில் ஒரு வார்த்தை தோன்றும் எண்ணிக்கைக்கு நிகரமாக அதிகரிக்கிறது, மேலும் அந்த வார்த்தை தொகுப்பில் உள்ள ஆவணங்களின் எண்ணிக்கையால் சமநிலைப்படுத்தப்படுகிறது, இது சில வார்த்தைகள் மற்றவற்றைவிட அடிக்கடி தோன்றும் உண்மையை சரிசெய்ய உதவுகிறது. உதாரணமாக, ஒரு வார்த்தை *ஒவ்வொரு* ஆவணத்திலும் தோன்றினால், $df_i=N$, மற்றும் $w_{ij}=0$, மேலும் அந்த வார்த்தைகள் முற்றிலும் புறக்கணிக்கப்படும்.

Scikit Learn பயன்படுத்தி நீங்கள் எளிதாக TF-IDF உரை வெக்டர்மயமாக்கலை உருவாக்கலாம்:


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

கேராஸில், `TextVectorization` லேயர் `output_mode='tf-idf'` பராமிட்டரை அனுப்புவதன் மூலம் TF-IDF அதிர்வெண்களை தானாக கணக்கிட முடியும். TF-IDF துல்லியத்தை அதிகரிக்கிறதா என்பதை பார்க்க மேலே பயன்படுத்திய குறியீட்டை மீண்டும் பயன்படுத்துவோம்:


In [17]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='tf-idf'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 12s 12ms/step - loss: 0.4197 - acc: 0.8662 - val_loss: 0.3432 - val_acc: 0.8849


## முடிவு

TF-IDF பிரதிநிதிகள் வெவ்வேறு சொற்களுக்கு அதிர்வெண் எடைகளை வழங்கினாலும், அவை அர்த்தத்தை அல்லது வரிசையை பிரதிநிதித்துவப்படுத்த முடியாது. பிரபலமான மொழியியல் நிபுணர் J. R. Firth 1935 ஆம் ஆண்டில் கூறியதுபோல், "ஒரு சொல்லின் முழுமையான அர்த்தம் எப்போதும் சூழலுடன் தொடர்புடையது, மற்றும் சூழலைத் தவிர்த்து அர்த்தத்தைப் பற்றிய எந்த ஆய்வும் முக்கியத்துவம் பெறாது." பாடநெறியின் பின்னர், மொழி மாதிரிகளைப் பயன்படுத்தி உரைமூலம் சூழலியல் தகவலை எவ்வாறு பிடிக்கலாம் என்பதை நாம் கற்றுக்கொள்வோம்.



---

**குறிப்பு**:  
இந்த ஆவணம் [Co-op Translator](https://github.com/Azure/co-op-translator) என்ற AI மொழிபெயர்ப்பு சேவையைப் பயன்படுத்தி மொழிபெயர்க்கப்பட்டுள்ளது. நாங்கள் துல்லியத்திற்காக முயற்சிக்கின்றோம், ஆனால் தானியங்கி மொழிபெயர்ப்புகளில் பிழைகள் அல்லது தவறான தகவல்கள் இருக்கக்கூடும் என்பதை தயவுசெய்து கவனத்தில் கொள்ளுங்கள். அதன் தாய்மொழியில் உள்ள மூல ஆவணம் அதிகாரப்பூர்வ ஆதாரமாக கருதப்பட வேண்டும். முக்கியமான தகவல்களுக்கு, தொழில்முறை மனித மொழிபெயர்ப்பு பரிந்துரைக்கப்படுகிறது. இந்த மொழிபெயர்ப்பைப் பயன்படுத்துவதால் ஏற்படும் எந்த தவறான புரிதல்கள் அல்லது தவறான விளக்கங்களுக்கு நாங்கள் பொறுப்பல்ல.
